In [294]:
import numpy as np
from sklearn.svm import SVR
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing


In [295]:
batting_data_path = 'bsb_ref.csv'
# INFO:
# 101,332 Players with up to 20 features each (exluding year, including team)
# if metric not reported for player, set to 0.0 by default

df_original = pd.read_csv(batting_data_path)
df = df_original.fillna(0) # fixing this could really help with results potentially
df_recent_players = df[df.year_ID >= 1975] 
df_probably_not_pitchers = df_recent_players[df_recent_players['PA'] >= 50]

# for the player specific SVR, I will choose players with at least 20 years of activity. 
# the data will be split as follows:
# x_train: year 0-5 data. the shape is: (num_samples, num_features) -> (6, num_features)
# y_train: year 6-12 OPS_plus. the shape is: (num_samples, 1) -> (6,1)

# x_test: year 13-14 data
# y_test: year 14-15 data
players = list(set(df_probably_not_pitchers['name_common'])) # just a set of all players

players_with_more_than_20_years = [player for player in players if len(df_probably_not_pitchers[df_probably_not_pitchers['name_common'] == player]) >= 20]
print(players_with_more_than_20_years)

['Tim Raines', 'Harold Baines', 'Dave Martinez', 'Adrian Beltre', 'Aramis Ramirez', 'Alex Rodriguez', 'Lenny Harris', 'Kenny Lofton', 'Jerry Hairston', 'Jose Cruz', 'Barry Bonds', 'Derek Jeter', 'Luis Gonzalez', 'Shawon Dunston', 'Andre Dawson', 'Carlos Beltran', 'Craig Biggio', 'Alex Gonzalez', 'Fred McGriff', 'Sandy Alomar', 'Julio Franco', 'Frank Thomas', 'Dave Winfield', 'Cal Ripken Jr.', 'Ivan Rodriguez', 'Tony Phillips', 'Ken Griffey', 'Jim Thome', 'Gary Matthews', 'Todd Zeile', 'Rickey Henderson', 'Omar Vizquel', 'Tony Pena', 'Jason Giambi', 'Tony Gwynn', 'Manny Ramirez', 'Steve Finley', 'B.J. Surhoff', 'Ruben Sierra', 'Tom Glavine', 'Ken Griffey Jr.', 'Gary Sheffield', 'Gary Gaetti', 'Paul Molitor', 'Eddie Murray', 'Rafael Palmeiro']


In [296]:
def prepare_test_train(player_x_data, player_y_data):
    i_0, i_1, i_2, i_3 = 6, 12, 14, 16 # hardcoded for now. these are just the ranges described above
    x_train = player_x_data[:i_0,:]
    y_train = player_y_data[i_0:i_1]
    x_test = player_x_data[i_1:i_2]
    y_test = player_y_data[i_2: i_3]
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    return (x_train, y_train, x_test, y_test)

In [297]:
def run_svr(data_tuple):
    x_train, y_train, x_test, y_test = data_tuple
    parameters = {
    "kernel": ["rbf"],
    "C": [1,10,10,100,1000],
    "gamma": [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
    }

    svr = GridSearchCV(SVR(), parameters, cv=5)
    svr.fit(x_train, y_train.ravel())

    y_svr_pred = svr.predict(x_test)
    print(y_test.ravel())
    print(y_svr_pred)

    print('Score: %.3f' % svr.score(x_train, y_train))
    print("Mean squared error: %.3f" % mean_squared_error(y_test, y_svr_pred))
    print('Mean Absolute error: %.3f' % mean_absolute_error(y_test, y_svr_pred))
    print('Variance score: %.3f' % r2_score(y_test, y_svr_pred))


    

In [298]:
def prepare_rows(player_rows):
    player_x_data =player_rows[['PA', 'G', 'Inn', 'runs_bat', 'runs_br', 'runs_dp', 'runs_field',
                                       'WAA', 'WAA_off', 'WAA_def', 'WAR', 'WAR_def', 'WAR_off', 'WAR_rep',
                                         'teamRpG', 'oppRpG', 'oppRpPA_rep', 'oppRpG_rep', 'OPS_plus']].values
    player_y_data = player_rows[['OPS_plus']].values
    return (preprocessing.scale(sandy_rows_np), preprocessing.scale(player_y_data))
    

In [299]:
for player in players_with_more_than_20_years:
    print(player)
    player_rows = df_probably_not_pitchers[df_probably_not_pitchers['name_common'] == player]
    data = prepare_rows(player_rows)
    data_tuple = prepare_test_train(data[0], data[1])
    run_svr(data_tuple)
    
    

Tim Raines
(6, 19) (6, 1) (2, 19) (2, 1)


/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[-0.23391573 -0.12815217]
[-0.01984985  0.16741433]
Score: 0.580
Mean squared error: 0.067
Mean Absolute error: 0.255
Variance score: -22.813
Harold Baines
(6, 19) (6, 1) (2, 19) (2, 1)


/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[-0.17332996  0.71362622]
[0.06980729 0.06980753]
Score: -0.117
Mean squared error: 0.237
Mean Absolute error: 0.443
Variance score: -0.204
Dave Martinez
(6, 19) (6, 1) (2, 19) (2, 1)


...

/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[1.12248034 0.86503894]
[0.37667968 0.37667925]
Score: -0.002
Mean squared error: 0.397
Mean Absolute error: 0.617
Variance score: -22.982
Eddie Murray
(6, 19) (6, 1) (2, 19) (2, 1)


/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[-0.73270374 -0.34547299]
[-0.27138419  0.80539539]
Score: 0.850
Mean squared error: 0.769
Mean Absolute error: 0.806
Variance score: -19.505
Rafael Palmeiro
(6, 19) (6, 1) (2, 19) (2, 1)
[0.47556638 0.6543033 ]
[0.22151617 0.22727028]
Score: -0.000
Mean squared error: 0.123
Mean Absolute error: 0.341
Variance score: -14.457


/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
